<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/post_tester(%EB%AC%B8%EC%84%9C_%EC%82%AC%ED%9B%84_%EA%B2%80%EC%A6%9D%EC%9A%A9)%2BKPI_%EC%9D%B8%EC%8B%9D_%EC%B2%AD%ED%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.8 MB/s eta 0:00:00


In [8]:
from __future__ import annotations
import re
import json
import hashlib
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional

# =============================================================
# 1. 청킹 핵심 로직 (Core Chunker Logic)
# =============================================================

# -----------------------------
# Regex patterns
# -----------------------------
MULTISPACE = re.compile(r'[ \t]+')
MULTINEW = re.compile(r'\n{3,}')
BULLET = re.compile(r'^\s*[-•▪◦·]\s*')

# KPI & number patterns (한국어 단위 포함)
PERCENT = r'\d{1,3}\s?%'
RANGE = r'\b\d{1,3}(?:\.\d+)?\s?[-–~]\s?\d{1,3}(?:\.\d+)?\b'
NUM_UNIT = r'\b\d{1,4}(?:,\d{3})*(?:\.\d+)?\s?(?:명|건|회|억|만원|천원|ms|초|분|시간|일|건수|건/일|건/월|건/년|GB|TB|건/초|req/s|bps|Mbps|GHz)\b'
TARGET_WORDS = r'(KPI|지표|목표|달성|성능|준수율|정확도|재현율|F1|응답\s?시간|가용성|오류율|완전성|오진률|매출|비용|절감|증가|감소|이상|이하|지원|동시\s?사용자|FDA|인증)'

NUMERIC_LINE = re.compile(
    rf'({TARGET_WORDS}).*?({PERCENT}|{NUM_UNIT}|{RANGE})',
    re.IGNORECASE
)

HEADER_LINE = re.compile(
    r'^\s*(?:#{1,6}\s+|[0-9]+[.)]\s+|[가-힣A-Z]+\s?[:：]|제\s?\d+\s?장|Ⅰ|Ⅱ|Ⅲ|IV|V)\s*.+'
)

# -----------------------------
# Utilities
# -----------------------------
def normalize_text(text: str) -> str:
    # 모든 유니코드 공백 문자(U+00A0 등)를 일반 공백(U+0020)으로 치환하여 SyntaxError 방지
    text = text.replace('\u00A0', ' ').replace('\r\n', '\n').replace('\r', '\n')
    text = MULTISPACE.sub(' ', text)
    text = MULTINEW.sub('\n\n', text)
    return text.strip()

def split_sentences(text: str) -> List[str]:
    parts = []
    # 문장 분리 후 마침표/물음표/느낌표 제거
    for para in text.split('\n'):
        para = para.strip()
        if not para:
            continue
        # 목록 기호 처리
        if BULLET.match(para):
            parts.append(BULLET.sub('', para).strip())
            continue
        # 문장 경계(. ! ? 。)를 기준으로 분리
        segs = re.split(r'(?<=[.!?。])\s+', para)
        parts.extend([s.strip() for s in segs if s.strip()])
    return parts

def hash_text(t: str) -> str:
    return hashlib.sha1(t.encode('utf-8')).hexdigest()[:12]

# -----------------------------
# Data structures & Dedup
# -----------------------------
@dataclass
class Chunk:
    id: str
    kind: str  # 'kpi' | 'header' | 'body'
    text: str
    meta: Dict[str, Any]

def dedup_chunks(chunks: List[Chunk], min_chars: int = 25) -> List[Chunk]:
    seen = set()
    out: List[Chunk] = []
    for ch in chunks:
        txt = ch.text.strip()
        if len(txt) < min_chars:
            continue
        key = hash_text(txt.lower())
        if key in seen:
            continue
        seen.add(key)
        out.append(ch)
    return out

# -----------------------------
# Window packing (Character-based)
# -----------------------------
def window_pack_chars(sentences: List[str], target_chars: int = 1200, overlap_chars: int = 220) -> List[str]:
    """문장 리스트를 목표 문자 수에 맞춰 오버랩되는 윈도우로 묶습니다."""
    windows = []
    buf, length = [], 0
    i = 0
    while i < len(sentences):
        s = sentences[i]

        # 새 문장을 추가해도 목표 길이 이내인 경우
        if not buf or length + len(s) + 1 <= target_chars:
            buf.append(s)
            length += len(s) + 1
            i += 1
        # 새 문장을 추가하면 목표 길이를 초과하는 경우
        else:
            windows.append(' '.join(buf).strip())

            # 오버랩을 위한 꼬리(tail)를 생성
            tail = ' '.join(buf)
            # 문자열의 뒤쪽 overlap_chars 만큼을 가져옵니다.
            tail = tail[-overlap_chars:] if len(tail) > overlap_chars else tail

            # 다음 윈도우의 시작으로 설정 (현재 문장은 다음 윈도우에 포함시키기 위해 i는 증가시키지 않음)
            buf, length = ([tail] if tail else []), len(tail)

    # 마지막 버퍼 처리
    if buf:
        windows.append(' '.join(buf).strip())

    # 긴 윈도우가 중복되어 잘렸을 경우를 대비하여 빈 문자열 제거 및 정제
    return [w for w in windows if w.strip()]

# -----------------------------
# Chunking core
# -----------------------------
def smart_chunk(text: str,
                source: str,
                target_chars: int = 1200,
                overlap_chars: int = 220,
                min_chars: int = 25) -> List[Chunk]:

    # 텍스트 정규화
    text = normalize_text(text)

    lines = [ln.strip() for ln in text.split('\n') if ln.strip()]
    headers, kpi_lines, body_lines = [], [], []

    # 1. 라인 분류 (KPI, Header, Body)
    for ln in lines:
        if NUMERIC_LINE.search(ln):
            kpi_lines.append(ln)
        elif HEADER_LINE.match(ln):
            headers.append(ln)
        else:
            body_lines.append(ln)

    # 2. KPI & Header 청크 생성 (Micro-chunks)
    kpi_chunks = [Chunk(id=f'kpi_{i}_{hash_text(ln)}', kind='kpi', text=ln, meta={'source': source})
                  for i, ln in enumerate(kpi_lines)]
    header_chunks = [Chunk(id=f'h_{i}_{hash_text(ln)}', kind='header', text=ln, meta={'source': source})
                     for i, ln in enumerate(headers)]

    # 3. Body 청크 생성 (Sliding Window)
    sentences = split_sentences('\n'.join(body_lines))

    windows = window_pack_chars(sentences, target_chars, overlap_chars)

    body_chunks = [Chunk(id=f'b_{i}_{hash_text(w)}', kind='body', text=w, meta={'source': source})
                   for i, w in enumerate(windows)]

    # 4. 통합 및 중복 제거
    chunks = kpi_chunks + header_chunks + body_chunks
    return dedup_chunks(chunks, min_chars=min_chars)

# -----------------------------
# Helper Functions (JSONL Save)
# -----------------------------
def save_jsonl(rows: List[Dict[str, Any]], path: str):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            # ensure_ascii=False는 한글 인코딩을 위해 필수
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

# =============================================================
# 2. 파일 내용 정의 (Fetched DOCX Content)
# =============================================================

# -- 1. eval_reports.docx 내용 (흉부 X-ray AI 진단 시스템) --
EVAL_REPORTS_DOCX = """
연구개발계획서
세부사업명: 산업기술R&D연구기획사업
연구개발 과제번호: 123456789
연구개발과제명: VUNO Med-Chest X-ray
전체 연구개발기간: 2023.12. ~
예산: 100,000 천원
연구개발 목표
연구개발 목표:VUNO Med-Chest X-ray의 최종 목표는 흉부 X-ray 판독의 정확도 및 효율성을 높이는 것이다. 구체적으로, 판독 정확도를 90% 이상으로 향상시키고, 판독 시간을 평균 30% 감소시키는 것을 목표로 한다. 이를 위해, 주요 흉부 소견(결절, 경화, 간질성 음영, 흉수, 기흉 등)의 자동 검출 및 위치 표시 기능을 강화하여 오진률을 최소화한다. KPI는 판독 정확도(%), 판독 시간(분), 오진률(%)로 설정하며, 검증 방법은 실제 임상 환경에서의 파일럿 테스트 및 사용자 피드백을 통해 이루어진다. 또한, 2024년 11월까지 FDA 510k 인증을 완료하고, 2025년까지 글로벌 시장 진출을 목표로 한다. 예산은 100,000천원으로, 이를 통해 기술적 개선 및 임상 검증을 수행하며, 최종적으로 공공의료기관 및 민간 의료기관에 솔루션을 제공하여 의료 서비스의 질을 향상시키는 데 기여한다.
연구개발 내용
## 연구개발 내용### 기술요소VUNO Med-Chest X-ray는 흉부 X-ray 영상에서 주요 소견을 자동으로 탐지 및 분류하는 AI 기반 시스템입니다.
핵심 기술 요소는 다음과 같습니다:- **딥러닝 모델**: CNN(Convolutional Neural Network) 기반으로 흉부 X-ray 영상 분석.- **데이터 증강**: 다양한 각도 및 조건에서의 X-ray 이미지 데이터 증강.- **특징 추출**: 결절, 경화, 간질성 음영, 흉수, 기흉 등 주요 소견의 특징 추출.- **후처리 알고리즘**: 탐지 결과의 정확도 향상을 위한 후처리 기법 적용.### 데이터- **학습 데이터**: 국내외 병원 및 의료기관에서 수집된 흉부 X-ray 영상 10,000건 이상.- **검증 데이터**: 독립적인 병원 3곳에서 수집된 2,000건 이상의 X-ray 영상.- **테스트 데이터**: 외부 기관 2곳에서 제공된 1,500건 이상의 X-ray 영상.### 시스템...
"""

# -- 2. RND_Plan(2).docx 내용 (분산형 임상연구 데이터 플랫폼) --
RND_PLAN2_DOCX = """
연구개발계획서
세부사업명: 디지털헬스케어사업
연구개발 과제번호: 2025-ABC-001
연구개발과제명: 분산형 임상연구 데이터 품질·표준화 플랫폼
전체 연구개발기간: 2025.01.01 ~ 2027.12.31
예산: 500000 천원
연구개발 목표
## 연구개발 목표### 디지털헬스케어사업 - 분산형 임상연구 데이터 품질·표준화 플랫폼**목표:**2025년 1월 1일부터 2027년 12월 31일까지, 본 과제는 분산형 임상연구 데이터의 품질 및 표준화를 위한 플랫폼을 개발한다.
총 예산은 500,000천원으로, 주요 목표는 다음과 같다:1. **데이터 품질 향상** - 데이터 오류율 5% 이하 유지 - 데이터 완전성 95% 이상 달성2.
**표준화 구현** - HL7 FHIR 표준 준수 - 10개 이상의 임상연구 유형 지원3.
**성능 지표** - 데이터 처리 속도: 1초당 1,000건 이상 - 시스템 가용성: 99.9% 이상**검증 방법:**- 내부 테스트: 100건 이상의 데이터 샘플로 품질 검증- 외부 검증: 3개 이상의 임상연구 기관 참여- 성능 테스트: 1,000건 이상의 데이터로 시스템 성능 확인**달성 기준:**- 1단계(2025년): 데이터 품질 80% 달성, 표준화 5개 유형 지원- 2단계(2026년): 데이터 품질 90%, 표준화 8개 유형 지원- 3단계(2027년): 최종 목표 달성 (품질 95%, 표준화 10개 유형)**핵심 키워드:** 데이터 품질, 표준화, HL7 FHIR, 성능 지표, 검증 방법.
연구개발 내용
## 목차사 업 보 고 서(제 11 기)【 대표이사 등의 확인 】대표집행임원 등의 확인 및 서명### I. 회사의 개요- **설립일자**: 2020년 1월 1일- **연결대상 종속회사**: 3개- **중소기업 해당 여부**: 해당- **주요사업 내용**: 디지털 헬스케어 솔루션 개발- **신용평가**: 해당사항 없음### II.
연구개발 내용#### 1. 연구개발 개요- **세부사업명**: 디지털헬스케어사업- **연구개발 과제번호**: 2025-ABC-001- **연구개발과제명**: 분산형 임상연구 데이터 품질·표준화 플랫폼- **기간**: 2025.01.01 ~ 2027.12.31- **예산**: 500,000,000원#### 2. 기술 요소- **데이터 표준화**: HL7, FHIR 표준 적용- **데이터 품질 관리**: 데이터 정제, 중복 제거, 유효성 검사- **분산형 아키텍처**: 블록체인 기반 데이터 저장 및 관리- **API 인터페이스**: RESTful API, GraphQL 지원#### 3. 서브태스크- **데이터 수집**: 다양한 임상 데이터 소스 통합- **데이터 정제**: 비정형 데이터 정형화- **표준화 모듈 개발**: HL7/FHIR 변환 로직 구현- **품질 검증**: 자동화된 테스트 케이스 설계...
"""

# -- 3. cde_연구계발계획서.docx 내용 (분산형 임상연구 데이터 플랫폼 - 상세 KPI) --
CDE_RND_DOCX = """
연구개발계획서
세부사업명: 디지털헬스케어사업
연구개발 과제번호: 2025-ABC-001
연구개발과제명: 분산형 임상연구 데이터 품질·표준화 플랫폼
전체 연구개발기간: 2025.01.01 ~ 2027.12.31
예산: 500000 천원
연구개발 목표
## 연구개발 목표### 1. 연구 기획 목표본 과제는 **분산형 임상연구 데이터 품질·표준화 플랫폼**을 개발하여, 2027년까지 다음과 같은 구체적인 목표를 달성합니다.- **데이터 품질 향상**: 임상연구 데이터의 오류율을 20% 감소시키고, 데이터 완전성(completeness)을 95% 이상으로 유지합니다.- **표준화 수준 강화**: 데이터 표준화 준수율을 90% 이상으로 설정하고, 표준화된 데이터 포맷(HL7 FHIR)을 100% 적용합니다.- **플랫폼 성능**: 플랫폼의 응답 시간(response time)을 500ms 이하로 유지하고, 동시 사용자 수 1,000명 이상을 지원합니다.### 2. 핵심 성능지표(KPI)- **데이터 오류율**: 20% 감소 (기준: 2025년 30%, 2027년 10%)- **데이터 완전성**: 95% 이상 유지- **표준화 준수율**: 90% 이상- **플랫폼 응답 시간**: 500ms 이하- **동시 사용자 수**: 1,000명 이상### 3. 달성 기준 및 검증 방법- **데이터 품질**: 연 2회 데이터 품질 평가(DQE) 실시, 오류율 및 완전성 지표 분석- **표준화 준수**: HL7 FHIR 표준 준수 테스트(연간 1회) 및 외부 인증 획득- **플랫폼 성능**: 성능 테스트(부하 테스트, 스트레스 테스트) 및 사용자 수용 테스트(UAT) 실시- **최종 검증**: 2027년 12월, 종합 성과 평가 및 외부 전문가 리뷰 진행### 4. 결론본 연구는 임상연구 데이터의 품질과 표준화를 혁신적으로 개선하여, 분산형 임상연구의 효율성을 극대화하는 것을 목표로 합니다.
이를 통해 의료 연구 및 임상 의사결정 지원 시스템의 신뢰성을 높이고, 궁극적으로 환자 치료의 질을 향상시키는 데 기여할 것입니다.
연구개발 내용
## 사업 보고서### 1. 연구개발 내용**과제명:** 분산형 임상연구 데이터 품질·표준화 플랫폼**기간:** 2025.01.01 ~ 2027.12.31**예산:** 500,000천원#### 1.1.
기술 요소본 과제는 임상연구 데이터의 분산 저장 및 품질 보장을 위한 플랫폼을 개발한다.
핵심 기술 요소는 다음과 같다:- **블록체인 기술:** 데이터의 무결성과 투명성을 보장.- **API 인터페이스:** 다양한 데이터 소스와의 연동을 지원.- **데이터 표준화 모듈:** HL7, FHIR 등 국제 표준을 준수.- **AI 기반 품질 검증:** 이상치 탐지 및 데이터 정합성 확인.#### 1.2.
서브태스크1. **데이터 수집 모듈 개발:** 다양한 임상 데이터 소스(전자건강기록, 웨어러블 기기 등)에서 데이터 수집.2.
**블록체인 네트워크 구축:** 데이터 저장 및 검증의 신뢰성 확보.3. **표준화 규칙 정의:** HL7, FHIR 표준에 따른 데이터 변환 규칙 수립.4.
**AI 품질 검증 시스템 구현:** 머신러닝 모델을 활용한 데이터 품질 평가.#### 1.3.
인터페이스- **API Gateway:** 외부 시스템과의 데이터 교환.- **사용자 인터페이스(UI):** 연구자가 데이터 품질 및 표준화를 모니터링.- **데이터 시각화 도구:** 수집된 데이터의 통계적 분석 제공.#### 1.4.
데이터/시스템 흐름1. **데이터 수집:** 다양한 소스에서 데이터 수집 → API Gateway를 통해 플랫폼에 전송.2.
**데이터 저장:** 블록체인 네트워크에 저장 → 무결성 검증.3. **표준화:** 표준화 모듈이 데이터 변환 → AI 품질 검증 시스템 적용.4.
**결과 제공:** 연구자에게 데이터 품질 보고서 및 시각화 결과 제공.#### 1.5. ...
"""

# -- 4. e5_연구계발계획서v6.docx 내용 (신제품 개발 및 매출 증가) --
E5_RND_V6_DOCX = """
연구개발계획서
세부사업명: 산업기술R&D연구기획사업
연구개발 과제번호: 123456789
연구개발과제명: VUNO Med-Chest X-ray
전체 연구개발기간: 2023. 6. 1 ~
예산: - 천원
연구개발 목표
---### 연구개발 목표**산업기술R&D연구기획사업****연구개발 과제번호: 123456789**#### 1. 연구개발 목표 정의본 연구개발 과제는 **신제품 개발**을 통해 **시장 점유율**을 확대하고자 합니다. 구체적으로는 **신제품 A**와 **신제품 B**의 개발을 완료하며, 각 제품의 **상용화 시점**은 **2024년 12월**로 설정합니다. 이를 통해 **연간 매출**을 **10% 증가**시키는 것을 목표로 합니다.#### 2. 핵심 성능지표(KPI)- **신제품 개발 완료율**: 100% (신제품 A, B 각각 100%)- **상용화 시기 준수율**: 100% (2024년 12월 내 상용화)- **매출 증가율**: 10% (2024년 대비 2025년 매출)#### 3. 달성 기준- **신제품 A**: 기능 테스트 완료 후 3개월 이내 상용화- **신제품 B**: 디자인 검토 완료 후 6개월 이내 상용화- **매출 증가**: 2025년 매출이 2024년 매출의 110% 이상 달성#### 4. 검증 방법- **신제품 개발 완료**: 내부 품질 검사 및 외부 전문가 평가- **상용화 시기 준수**: 프로젝트 관리 시스템(PMS)을 통한 일정 관리 및 정기 보고- **매출 증가**: 분기별 매출 보고서 분석 및 비교#### 5.
결론위 목표를 달성하기 위해 체계적인 프로젝트 관리와 지속적인 성과 모니터링이 필수적입니다.
각 단계별 마일스톤을 명확히 설정하고, 정기적인 검토를 통해 목표 달성을 위한 최적의 전략을 수립할 것입니다.
연구개발 내용
---### 연구개발 내용#### 1. 연구개발 목표본 연구개발 과제는 **스마트 제조 시스템** 구축을 목표로 한다. 이를 통해 생산 공정의 효율성을 극대화하고, 실시간 데이터 분석을 통한 품질 관리 체계를 강화한다. 목표는 다음과 같다:- 생산 라인의 자동화 수준 향상- 실시간 모니터링 시스템 구축- 데이터 기반 의사결정 지원 체계 마련#### 2. 기술 요소- **IoT 센서**: 생산 설비에 설치되어 실시간 데이터를 수집한다.- **데이터 분석 플랫폼**: 수집된 데이터를 분석하여 유의미한 인사이트를 도출한다.- **클라우드 컴퓨팅**: 대용량 데이터를 저장하고 처리하기 위한 인프라로 활용된다.- **AI 알고리즘**: 예측 유지보수 및 품질 검사에 적용된다.#### 3. 서브태스크1.
**IoT 센서 설치 및 네트워크 구성** - 생산 설비에 IoT 센서를 설치하고, 안정적인 통신 네트워크를 구축한다. - 데이터 전송 속도: 최소 100Mbps 이상2. **데이터 수집 및 전처리** - 센서로부터 수집된 데이터를 정제하고, 분석에 적합한 형태로 변환한다. - 데이터 포맷: JSON, CSV3. **데이터 분석 및 시각화** - 수집된 데이터를 기반으로 실시간 대시보드를 개발한다. - 분석 주기: 1분 단위4. **AI 모델 개발 및 적용** - 예측 유지보수 및 품질 검사 모델을 개발하고, 실제 데이터에 적용한다. - 모델 성능 지표: 정확도 90% 이상#### 4. 인터페이스- **센서 ↔ 데이터 수집 서버**: MQTT 프로토콜 사용- **데이터 수집 서버 ↔ 클라우드**: REST API 활용- **클라우드 ↔ 사용자 인터페이스**: 웹 기반 대시보드 제공#### 5. 데이터/시스템 흐름1.
**데이터 수집 단계** - IoT 센서가 생산 설비에서 데이터를 수집 - 데이터는 MQTT 프로토콜을 통해 데이터 수집 서버로 전송2.
**데이터 처리 단계** - 데이터 수집 서버에서 데이터 정제 및 전처리 - 정제된 데이터는 클라우드로 전송3.
**분석 및 시각화 단계** - 클라우드에서 AI 알고리즘이 데이터를 분석 - 분석 결과는 웹 대시보드로 시각화4.
**피드백 루프** - 분석 결과를 바탕으로 생산 공정 개선 방안 도출 - 개선 사항은 다시 IoT 센서에 피드백되어 지속적인 최적화 진행####...
"""


# =============================================================
# 3. 통합 실행 함수
# =============================================================

def execute_chunking():
    # 청킹 매개변수 설정
    params = {
        "target_chars": 900,
        "overlap_chars": 180,
        "min_chars": 50,
        "out": "chunks_final_corpus.jsonl"
    }

    files_to_chunk = [
        ("eval_reports.docx", EVAL_REPORTS_DOCX),
        ("RND_Plan(2).docx", RND_PLAN2_DOCX),
        ("cde_연구계발계획서.docx", CDE_RND_DOCX),
        ("e5_연구계발계획서v6.docx", E5_RND_V6_DOCX),
    ]

    final_corpus: List[Dict[str, Any]] = []

    print("--- 📝 KPI-aware 통합 청킹 실행 ---")

    # 3. 각 파일에 대해 청킹 실행
    for filename, content in files_to_chunk:
        print(f"[INFO] 📄 파일 처리: {filename}")

        if content:
            chunks = smart_chunk(
                content, # 전처리 없이 normalize_text는 smart_chunk 내부에서 처리
                source=filename,
                target_chars=params["target_chars"],
                overlap_chars=params["overlap_chars"],
                min_chars=params["min_chars"]
            )

            print(f"[INFO] -> {len(chunks)}개의 청크 생성 완료.")
            final_corpus.extend(asdict(ch) for ch in chunks)
        else:
            print("[WARN] 파일 내용이 비어있습니다. 청크 0개.")

    # 4. JSONL로 저장 (출력은 JSONL 문자열로 시뮬레이트)
    output_jsonl_preview = ""
    for i, row in enumerate(final_corpus):
        if i < 3:
            # 처음 3개만 미리보기
            output_jsonl_preview += json.dumps(row, ensure_ascii=False) + "\n"
        elif i == 3:
            output_jsonl_preview += f"... (총 {len(final_corpus)}개의 청크 중 나머지 생략)\n"
            break

    if len(final_corpus) > 0 and len(final_corpus) <= 3:
        output_jsonl_preview = "".join(json.dumps(row, ensure_ascii=False) + "\n" for row in final_corpus)

    print(f"\n=============================================")
    print(f"✅ 청킹 완료! 총 {len(final_corpus)}개의 청크가 생성되었습니다.")
    print(f"   (결과는 '{params['out']}' 파일로 저장될 예정이며, 다음은 미리보기입니다.)")
    print("=============================================")
    print(output_jsonl_preview)

if __name__ == "__main__":
    execute_chunking()

--- 📝 KPI-aware 통합 청킹 실행 ---
[INFO] 📄 파일 처리: eval_reports.docx
[INFO] -> 4개의 청크 생성 완료.
[INFO] 📄 파일 처리: RND_Plan(2).docx
[INFO] -> 5개의 청크 생성 완료.
[INFO] 📄 파일 처리: cde_연구계발계획서.docx
[INFO] -> 4개의 청크 생성 완료.
[INFO] 📄 파일 처리: e5_연구계발계획서v6.docx
[INFO] -> 3개의 청크 생성 완료.

✅ 청킹 완료! 총 16개의 청크가 생성되었습니다.
   (결과는 'chunks_final_corpus.jsonl' 파일로 저장될 예정이며, 다음은 미리보기입니다.)
{"id": "kpi_0_71acd27fbd9e", "kind": "kpi", "text": "연구개발 목표:VUNO Med-Chest X-ray의 최종 목표는 흉부 X-ray 판독의 정확도 및 효율성을 높이는 것이다. 구체적으로, 판독 정확도를 90% 이상으로 향상시키고, 판독 시간을 평균 30% 감소시키는 것을 목표로 한다. 이를 위해, 주요 흉부 소견(결절, 경화, 간질성 음영, 흉수, 기흉 등)의 자동 검출 및 위치 표시 기능을 강화하여 오진률을 최소화한다. KPI는 판독 정확도(%), 판독 시간(분), 오진률(%)로 설정하며, 검증 방법은 실제 임상 환경에서의 파일럿 테스트 및 사용자 피드백을 통해 이루어진다. 또한, 2024년 11월까지 FDA 510k 인증을 완료하고, 2025년까지 글로벌 시장 진출을 목표로 한다. 예산은 100,000천원으로, 이를 통해 기술적 개선 및 임상 검증을 수행하며, 최종적으로 공공의료기관 및 민간 의료기관에 솔루션을 제공하여 의료 서비스의 질을 향상시키는 데 기여한다.", "meta": {"source": "eval_reports.docx"}}
{"id": "kpi_1_035901068a9d", "kind": "kpi", "text": "핵심 기술 요소는 다음과 같습니다:- **딥러닝 모델**: C

In [9]:
%%writefile doc_eval_refactored.py
# -*- coding: utf-8 -*-
"""
Doc Evaluator (Refactored)
- 문서별 Claim 자동추출 (숫자·단위 KPI, 비율, 마감/기간 등 룰 기반)
- 하이브리드 검색: BM25 + 임베딩(BGE-M3). 임베딩/라이브러리 없으면 BM25로 폴백
- NLI + Boolean QA 결합 판정 (임계값/우선순위 적용)
- Major/Minor 지표 '실제' 집계 (고정값/더미 제거)
- 한국어 문장 분할 개선, heading 기반 섹션 추출, 형식 점검 개선
"""

import os, re, json, math
from typing import List, Dict, Any, Tuple
from collections import namedtuple, Counter
import docx

_EMBED_OK = False
_BM25_OK = False
try:
    from sentence_transformers import SentenceTransformer
    import numpy as np
    _EMBED_OK = True
except Exception:
    _EMBED_OK = False

try:
    from rank_bm25 import BM25Okapi
    _BM25_OK = True
except Exception:
    _BM25_OK = False


class DocParser:
    def parse(self, docx_path: str):
        try:
            d = docx.Document(docx_path)
        except Exception:
            return None

        paras = [p.text.strip() for p in d.paragraphs if p.text and p.text.strip()]

        # Heading 기반 섹션 추출
        sections = []
        cur_title = None
        cur_buf = []
        for p in d.paragraphs:
            style = getattr(p.style, "name", "") or ""
            text = p.text.strip()
            if not text:
                continue
            if style.startswith("Heading") or "제목" in style:
                # 섹션 마감
                if cur_title or cur_buf:
                    sections.append({"title": cur_title, "text": "\n".join(cur_buf)})
                cur_title = text
                cur_buf = []
            else:
                cur_buf.append(text)
        if cur_title or cur_buf:
            sections.append({"title": cur_title, "text": "\n".join(cur_buf)})

        full_text = "\n".join(paras)
        sentences = split_ko_sentences(full_text)

        Doc = namedtuple("Doc", ["sections", "paragraphs", "sentences", "text"])
        return Doc(sections=sections, paragraphs=paras, sentences=sentences, text=full_text)


def split_ko_sentences(text: str) -> List[str]:
    # 문장 종결부호 + '다.' 패턴 반영
    # 공백/줄바꿈 기준으로 재조합
    text = re.sub(r"\s+", " ", text)
    sents = re.split(r"(?<=[\.?!])\s+|(?<=다\.)\s+", text)
    sents = [s.strip() for s in sents if s and s.strip()]
    return sents


KPI_PATTERNS = [
    r"\b\d{1,3}\s?%(\s?(감소|증가|유지))",     # 20% 감소/증가/유지
    r"(오류율|에러율)\s*\d{1,3}\s?%",         # 오류율 20%
    r"(정확도|완전성|재현율|정밀도)\s*(\d{1,3}\s?%)",  # 정확도 95%
    r"(응답시간|지연)\s*\d+(\.\d+)?\s?(ms|초)",       # 응답시간 500ms
    r"(처리량|TPS|QPS)\s*\d+(\.\d+)?",               # TPS 1000
    r"(기간|마감|데드라인)\s*(\d+\s?(일|주|개월|월|분기|년))",  # 기간 6개월
    r"(비용|원가)\s*\d+(,\d{3})*(\.\d+)?\s?(원|만원|억)",        # 비용 1,000만원
]

def extract_claims(doc) -> List[Dict[str, Any]]:
    claims = []
    for sent in doc.sentences:
        hit = False
        for pat in KPI_PATTERNS:
            if re.search(pat, sent):
                hit = True
                break
        if hit:
            claims.append({"sent": sent, "type": "KPI"})
    # 너무 적으면 상위 문장(제목/요약 추정)에서도 보완 수집
    if len(claims) < 5:
        head_boost = doc.paragraphs[:10]
        for s in head_boost:
            if any(re.search(p, s) for p in KPI_PATTERNS):
                claims.append({"sent": s, "type": "KPI"})
    # 중복 제거
    uniq = []
    seen = set()
    for c in claims:
        if c["sent"] not in seen:
            uniq.append(c); seen.add(c["sent"])
    return uniq[:15] if uniq else []


class HybridRetriever:
    def __init__(self, cfg: Dict[str, Any], corpus_texts: List[str]):
        self.cfg = cfg
        self.corpus = corpus_texts or []
        self.use_embed = False
        self.use_bm25  = False

        # BM25
        if _BM25_OK and self.corpus:
            tokenized = [self.tokenize(x) for x in self.corpus]
            self.bm25 = BM25Okapi(tokenized)
            self.use_bm25 = True

        # 임베딩
        self.embed_dim = None
        if _EMBED_OK and self.corpus and cfg["models"].get("embed"):
            try:
                self.embed = SentenceTransformer(cfg["models"]["embed"])
                self.corpus_vec = self.embed.encode(self.corpus, normalize_embeddings=True)
                self.embed_dim = self.corpus_vec.shape[1]
                self.use_embed = True
            except Exception:
                self.use_embed = False

    @staticmethod
    def tokenize(x: str) -> List[str]:
        return re.findall(r"[가-힣A-Za-z0-9]+", x.lower())

    def topk(self, query: str, k: int = 5) -> List[Tuple[str, float]]:
        cand: Dict[int, float] = {}

        # BM25 스코어
        if self.use_bm25:
            scores = self.bm25.get_scores(self.tokenize(query))
            for i, s in enumerate(scores):
                if s > 0:
                    cand[i] = cand.get(i, 0.0) + float(s)

        # 임베딩 스코어(코사인)
        if self.use_embed:
            qv = self.embed.encode([query], normalize_embeddings=True)[0]
            sims = (self.corpus_vec @ qv)
            for i, s in enumerate(sims):
                cand[i] = cand.get(i, 0.0) + float(s) * 100.0  # 임베딩 가중(스케일 정규화 목적)

        # 둘 다 실패한 경우: 부분문자열 폴백
        if not cand:
            for i, t in enumerate(self.corpus):
                if query[:20] in t:
                    cand[i] = 1.0

        ranked = sorted(cand.items(), key=lambda x: x[1], reverse=True)[:k]
        return [(self.corpus[i], score) for i, score in ranked]


class NLIModel:
    """
    외부 대형 NLI 모델이 없는 환경에서도 동작하도록 규칙 기반 점수 제공.
    - 숫자/단위 일치도가 높으면 entailment를, 상충 수치가 발견되면 contradiction을 강화
    """
    def __init__(self, cfg):
        self.cfg = cfg

    def predict(self, claim: str, evidence: str) -> Dict[str, float]:
        claim_nums = extract_numbers_units(claim)
        evid_nums  = extract_numbers_units(evidence)

        # 기본 확률
        entail = 0.33; contra = 0.33; neutr = 0.34
        match_cnt, conflict = number_match_quality(claim_nums, evid_nums)

        if match_cnt > 0 and not conflict:
            entail = 0.7; contra = 0.1; neutr = 0.2
        elif conflict:
            entail = 0.1; contra = 0.7; neutr = 0.2
        else:
            # 유사 텍스트 힌트
            if keyword_overlap(claim, evidence) > 0.4:
                entail = 0.5; neutr = 0.4; contra = 0.1

        max_p = max(entail, contra, neutr)
        return {"entail": entail, "contra": contra, "neutral": neutr, "max_p": max_p}


class BooleanQA:
    """아주 짧은 yes/no 룰베이스. (실모델 없을 때 폴백)"""
    def __init__(self, cfg):
        self.cfg = cfg

    def yesno(self, question: str, context: str) -> str:
        # 키워드 중첩/수치 매칭이 일정 이상이면 yes
        q = question.lower()
        ko = keyword_overlap(q, context)
        cn, en = extract_numbers_units(q), extract_numbers_units(context)
        match_cnt, conflict = number_match_quality(cn, en)
        if conflict:
            return "no"
        if match_cnt >= 1 or ko > 0.35:
            return "yes"
        return "no"

def extract_numbers_units(text: str) -> List[Tuple[float, str]]:
    out = []
    # 20%, 95 %, 500ms, 10개월, 1,000만원 등
    for m in re.finditer(r"(\d+(?:[\.,]\d+)?)(\s?%|ms|초|일|주|개월|월|분기|년|원|만원|억)?", text):
        num = m.group(1).replace(",", "")
        unit = (m.group(2) or "").strip()
        try:
            out.append((float(num), unit))
        except Exception:
            pass
    return out

def number_match_quality(a: List[Tuple[float,str]], b: List[Tuple[float,str]]) -> Tuple[int, bool]:
    """같은 단위에서 숫자가 근접하면 match, 큰 차이면 conflict로."""
    match = 0; conflict = False
    for ax, au in a:
        for bx, bu in b:
            if au and bu and au == bu:
                # 근사 일치(상대 오차 10% 이내)
                if bx == 0:
                    continue
                rerr = abs(ax - bx) / (abs(bx) + 1e-6)
                if rerr <= 0.1:
                    match += 1
                elif rerr >= 0.5:
                    conflict = True
    return match, conflict

def keyword_overlap(a: str, b: str) -> float:
    ta = set(re.findall(r"[가-힣A-Za-z0-9]+", a.lower()))
    tb = set(re.findall(r"[가-힣A-Za-z0-9]+", b.lower()))
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

def ngram_redundancy(sentences: List[str], n: int = 3) -> float:
    """3-gram 중복 비율"""
    grams = []
    for s in sentences:
        toks = re.findall(r"[가-힣A-Za-z0-9]+", s.lower())
        grams += list(zip(*[toks[i:] for i in range(n)]))
    if not grams:
        return 0.0
    c = Counter(grams)
    dup = sum(v-1 for v in c.values() if v>1)
    return dup / (len(grams) + 1e-6)

def simple_coherence(sentences: List[str]) -> float:
    """아주 단순한 연결성: 인접 문장 키워드 교집합 비율 평균"""
    if len(sentences) < 2:
        return 0.5
    scores = []
    for i in range(len(sentences)-1):
        scores.append(keyword_overlap(sentences[i], sentences[i+1]))
    return sum(scores)/len(scores)

def format_score(sections: List[Dict[str,str]], required_titles: List[str]) -> float:
    titles = [ (s["title"] or "").strip() for s in sections if s.get("title") ]
    hit = 0
    for req in required_titles:
        # 부분 일치 허용
        if any(req in (t or "") for t in titles):
            hit += 1
    if not required_titles:
        return 1.0
    return hit / len(required_titles)

def aggregate_scores(major: Dict[str, float], minor: Dict[str, float], glossary_on: bool) -> Dict[str, float]:
    # terminology는 glossary 있을 때만 반영
    term_w = 0.1 if glossary_on else 0.0
    norm_w = 1.0 - term_w
    # minor 합성
    minor_core = (
        0.4 * minor["coherence"] +
        0.3 * minor["fluency"] +
        0.2 * (1 - minor["redundancy"]) +
        0.1 * minor["format"]
    )
    minor_total = norm_w * minor_core + term_w * minor.get("terminology", 0.0)
    final = 0.6 * major["accuracy"] + 0.4 * minor_total
    return {"final_score": final}

def build_report(doc, results, major, minor, final, claimed_top=10) -> str:
    rep = []
    rep.append("## 문서 평가 보고서\n")
    rep.append(f"### 최종 점수: {final['final_score']:.3f}\n")
    rep.append("### 주요 오류 분석 (Major)\n")
    rep.append(f"- Accuracy: {major['accuracy']:.3f} (entail={major['entail_cnt']}, contra={major['contra_cnt']}, unknown={major['unknown_cnt']})\n")
    if "precision" in major:
        rep.append(f"- Precision: {major['precision']:.3f}, Recall: {major['recall']:.3f}, F1: {major['f1']:.3f}\n")
    rep.append("\n#### 개별 주장 검증 상위 결과\n")
    for r in results[:claimed_top]:
        rep.append(f"- 주장: {r['sent']}\n")
        best_txt = r['best_evidence'][:300].replace("\n", " ")
        rep.append(f"  - 근거: {best_txt} ...\n")
        rep.append(f"  - 판정: {r['verdict']} (신뢰도: {r['confidence']:.2f})\n\n")

    rep.append("### 사소 오류 분석 (Minor)\n")
    rep.append(f"- Fluency: {minor['fluency']:.3f}\n")
    rep.append(f"- Coherence: {minor['coherence']:.3f}\n")
    rep.append(f"- Redundancy(낮을수록 좋음): {minor['redundancy']:.3f}\n")
    rep.append(f"- Format: {minor['format']:.3f}\n")
    if "terminology" in minor:
        rep.append(f"- Terminology: {minor['terminology']:.3f}\n")

    # 개선 제안
    rep.append("\n### 개선 제안\n")
    if major["contra_cnt"] > 0 or major["unknown_cnt"] > 0:
        rep.append("- KPI에 대한 정확한 수치/근거 인용을 본문/부록에 명시하세요(표·각주·참고문헌 라벨).\n")
    if minor["redundancy"] > 0.25:
        rep.append("- 중복 문장을 통합하세요(요약/표로 치환, 불필요한 반복 제거).\n")
    if minor["format"] < 0.8:
        rep.append("- 필수 섹션(연구개발 목표/내용/기대효과/추진체계/활용계획 등)의 제목을 명시하고 순서를 정리하세요.\n")
    rep.append("- 섹션 간 연결어(따라서, 한편, 결과적으로 등)를 활용해 응집성을 높이세요.\n")

    return "".join(rep)

# 평가기
class DocEvaluator:
    def __init__(self, cfg: Dict[str, Any]):
        self.cfg = cfg
        self.parser = DocParser()
        self.nli = NLIModel(cfg["models"].get("nli"))
        self.boolqa = BooleanQA(cfg["models"].get("qna"))
        # 용어집
        self.glossary = self._load_glossary(cfg["inputs"].get("domain_glossary"))

    def _load_glossary(self, file_path: str):
        if not file_path or not os.path.exists(file_path):
            return None
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return None

    def evaluate(self, docx_path: str) -> Dict[str, Any]:
        doc = self.parser.parse(docx_path)
        if not doc:
            return {"final_score": 0.0, "error": "Document parsing failed"}

        # 코퍼스(근거 후보): 문장 + 섹션 텍스트
        corpus = doc.sentences[:]
        for s in doc.sections:
            corpus.append(s["text"])
        retriever = HybridRetriever(self.cfg, corpus)

        # 1) Claim 추출
        claims = extract_claims(doc)
        if not claims:
            # 주장 없으면 문서 핵심문장(상위 10개)로 대체
            base = [{"sent": s, "type": "fallback"} for s in doc.sentences[:10]]
            claims = base

        # 2) 검증
        results = []
        entail_cnt = contra_cnt = unknown_cnt = 0
        for c in claims:
            top = retriever.topk(c["sent"], k=5)
            evidences = [t for t, sc in top]
            best = evidences[0] if evidences else ""
            nli = self.nli.predict(c["sent"], best)
            verdict, conf = self._post_decide(c["sent"], best, nli)

            if verdict == "entailment": entail_cnt += 1
            elif verdict == "contradiction": contra_cnt += 1
            else: unknown_cnt += 1

            results.append({
                **c,
                "best_evidence": best,
                "verdict": verdict,
                "confidence": conf
            })

        # 3) Major 지표
        tot = max(1, (entail_cnt + contra_cnt + unknown_cnt))
        accuracy = entail_cnt / tot
        # QA 기반 precision/recall 추정(간단 정의): yes=정답, no=오답 가정
        tp = entail_cnt
        fp = contra_cnt
        fn = unknown_cnt
        precision = tp / (tp + fp + 1e-6)
        recall    = tp / (tp + fn + 1e-6)
        f1        = 2*precision*recall / (precision+recall+1e-6)
        major_scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "entail_cnt": entail_cnt,
            "contra_cnt": contra_cnt,
            "unknown_cnt": unknown_cnt
        }

        # 4) Minor 지표
        fluency = self._fluency(doc.sentences)
        coherence = simple_coherence(doc.sentences)
        redundancy = ngram_redundancy(doc.sentences, n=3)
        req_titles = self.cfg.get("format_required", [
            "연구개발 목표", "연구개발 내용", "연구개발성과 활용계획", "추진체계", "기대효과"
        ])
        fmt = format_score(doc.sections, req_titles)
        terminology = self._terminology_score(doc.text)
        minor_scores = {
            "fluency": float(fluency),
            "coherence": float(coherence),
            "redundancy": float(redundancy),
            "format": float(fmt)
        }
        if terminology is not None:
            minor_scores["terminology"] = float(terminology)

        # 5) 최종 집계
        final = aggregate_scores(major_scores, minor_scores, glossary_on=(terminology is not None))

        # 6) 리포트 저장
        file_name = os.path.basename(docx_path).rsplit(".docx", 1)[0] + "_report.txt"
        report = build_report(doc, results, major_scores, minor_scores, final)
        os.makedirs(self.cfg["output_dir"], exist_ok=True)
        save_path = os.path.join(self.cfg["output_dir"], file_name)
        with open(save_path, "w", encoding="utf-8") as f:
            f.write(report)

        return {"final_score": final["final_score"], "report_path": save_path,
                "major": major_scores, "minor": minor_scores}

    def _post_decide(self, claim: str, evidence: str, nli_out: Dict[str,float]) -> Tuple[str, float]:
        # 임계값
        conf = nli_out["max_p"]
        if conf >= 0.65:
            # NLI 신뢰 높음 → 그대로
            return self._argmax_verdict(nli_out), conf

        # 신뢰 낮음 → QA로 보정
        qa_ans = self.boolqa.yesno(f"Is the claim supported? {claim}", evidence)
        if qa_ans.lower().startswith("y"):
            # entailment로 보정
            return "entailment", max(0.65, conf)
        else:
            # contradiction 우선
            return "contradiction", max(0.65, conf)

    @staticmethod
    def _argmax_verdict(nli_out: Dict[str,float]) -> str:
        m = max((("entailment", nli_out["entail"]),
                 ("contradiction", nli_out["contra"]),
                 ("neutral", nli_out["neutral"])), key=lambda x:x[1])[0]
        return m

    def _fluency(self, sents: List[str]) -> float:
        # 간이 유창성: 평균 문장 길이·기호 비율을 바탕으로 0~1 스코어
        if not sents:
            return 0.5
        lens = [len(s) for s in sents]
        mean_len = sum(lens)/len(lens)
        punct = sum(ch in ".,;:?!~" for s in sents for ch in s) / (sum(lens)+1e-6)
        # 경험적 매핑
        score = 0.5 + 0.5 * math.tanh((mean_len-25)/50) - 0.2*abs(punct-0.03)
        return max(0.0, min(1.0, score))

    def _terminology_score(self, text: str):
        if not self.glossary:
            return None
        terms = set(self.glossary.get("terms", []))
        if not terms:
            return None
        toks = set(re.findall(r"[가-힣A-Za-z0-9\-]+", text))
        hit = len(terms & toks)
        return hit / len(terms)

# 실행
# ======================
config = {
    "models": {"embed": "BAAI/bge-m3", "nli": "rule-lite", "qna": "rule-lite"},
    "inputs": {"domain_glossary": None},
    "format_required": [
         "연구개발 목표",
    "연구개발 내용",
    "연구개발성과 활용계획 및 기대효과",
    "연구기획과제의 개요",
    "연구개발과제의 배경",
    "연구개발과제의 필요성",
    "보안등급의 분류 및 해당 사유",
    "기술개발 핵심어(키워드)",
    "연차별 개발목표",
    "연차별 개발내용 및 범위",
    "추진방법 및 전략",
    "과제 성과의 활용방안",
    "신규사업 신설의 기대효과",
    "사회적 가치 창출 계획",
    "사회적 가치창출의 기대효과",
    "경제적 성과창출의 기대효과",
    "신규 인력 채용 계획 및 활용 방안"
    ],
    "output_dir": "./eval_reports"
}


Writing doc_eval_refactored.py


In [10]:
import importlib
import doc_eval_refactored
importlib.reload(doc_eval_refactored)

from doc_eval_refactored import DocEvaluator

In [11]:
config = {
    "models": {"embed": "BAAI/bge-m3", "nli": "rule-lite", "qna": "rule-lite"},
    "inputs": {"domain_glossary": None},
    "format_required": [
         "연구개발 목표",
        "연구개발 내용",
        "연구개발성과 활용계획 및 기대효과",
        "연구기획과제의 개요",
        "연구개발과제의 배경",
        "연구개발과제의 필요성",
        "보안등급의 분류 및 해당 사유",
        "기술개발 핵심어(키워드)",
        "연차별 개발목표",
        "연차별 개발내용 및 범위",
        "추진방법 및 전략",
        "과제 성과의 활용방안",
        "신규사업 신설의 기대효과",
        "사회적 가치 창출 계획",
        "사회적 가치창출의 기대효과",
        "경제적 성과창출의 기대효과",
        "신규 인력 채용 계획 및 활용 방안"
    ],
    "output_dir": "./eval_reports"
}

ev = DocEvaluator(config)
result = ev.evaluate("/content/RND_Plan(2).docx")
print("최종 점수:", result["final_score"])
print("리포트 저장:", result["report_path"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

최종 점수: 0.5450576631121106
리포트 저장: ./eval_reports/RND_Plan(2)_report.txt


In [12]:
config = {
    "models": {"embed": "BAAI/bge-m3", "nli": "rule-lite", "qna": "rule-lite"},
    "inputs": {"domain_glossary": None},
    "format_required": [
         "연구개발 목표",
    "연구개발 내용",
    "연구개발성과 활용계획 및 기대효과",
    "연구기획과제의 개요",
    "연구개발과제의 배경",
    "연구개발과제의 필요성",
    "보안등급의 분류 및 해당 사유",
    "기술개발 핵심어(키워드)",
    "연차별 개발목표",
    "연차별 개발내용 및 범위",
    "추진방법 및 전략",
    "과제 성과의 활용방안",
    "신규사업 신설의 기대효과",
    "사회적 가치 창출 계획",
    "사회적 가치창출의 기대효과",
    "경제적 성과창출의 기대효과",
    "신규 인력 채용 계획 및 활용 방안"
    ],
    "output_dir": "./eval_reports"
}

ev = DocEvaluator(config)
result = ev.evaluate("/content/cde_연구계발계획서.docx")
print("최종 점수:", result["final_score"])
print("리포트 저장:", result["report_path"])

최종 점수: 0.5191954306944678
리포트 저장: ./eval_reports/cde_연구계발계획서_report.txt


In [13]:
config = {
    "models": {"embed": "BAAI/bge-m3", "nli": "rule-lite", "qna": "rule-lite"},
    "inputs": {"domain_glossary": None},
    "format_required": [
         "연구개발 목표",
    "연구개발 내용",
    "연구개발성과 활용계획 및 기대효과",
    "연구기획과제의 개요",
    "연구개발과제의 배경",
    "연구개발과제의 필요성",
    "보안등급의 분류 및 해당 사유",
    "기술개발 핵심어(키워드)",
    "연차별 개발목표",
    "연차별 개발내용 및 범위",
    "추진방법 및 전략",
    "과제 성과의 활용방안",
    "신규사업 신설의 기대효과",
    "사회적 가치 창출 계획",
    "사회적 가치창출의 기대효과",
    "경제적 성과창출의 기대효과",
    "신규 인력 채용 계획 및 활용 방안"
    ],
    "output_dir": "./eval_reports"
}

ev = DocEvaluator(config)
result = ev.evaluate("/content/e5_연구계발계획서v6.docx")
print("최종 점수:", result["final_score"])
print("리포트 저장:", result["report_path"])

최종 점수: 0.7029831838656648
리포트 저장: ./eval_reports/e5_연구계발계획서v6_report.txt


In [14]:
config = {
    "models": {"embed": "BAAI/bge-m3", "nli": "rule-lite", "qna": "rule-lite"},
    "inputs": {"domain_glossary": None},
    "format_required": [
         "연구개발 목표",
    "연구개발 내용",
    "연구개발성과 활용계획 및 기대효과",
    "연구기획과제의 개요",
    "연구개발과제의 배경",
    "연구개발과제의 필요성",
    "보안등급의 분류 및 해당 사유",
    "기술개발 핵심어(키워드)",
    "연차별 개발목표",
    "연차별 개발내용 및 범위",
    "추진방법 및 전략",
    "과제 성과의 활용방안",
    "신규사업 신설의 기대효과",
    "사회적 가치 창출 계획",
    "사회적 가치창출의 기대효과",
    "경제적 성과창출의 기대효과",
    "신규 인력 채용 계획 및 활용 방안"
    ],
    "output_dir": "./eval_reports"
}

ev = DocEvaluator(config)
result = ev.evaluate("/content/eval_reports.docx")
print("최종 점수:", result["final_score"])
print("리포트 저장:", result["report_path"])

최종 점수: 0.2195769514399627
리포트 저장: ./eval_reports/eval_reports_report.txt


In [15]:
!zip -r my_results.zip /content/eval_reports

  adding: content/eval_reports/ (stored 0%)
  adding: content/eval_reports/RND_Plan(2)_report.txt (deflated 62%)
  adding: content/eval_reports/cde_연구계발계획서_report.txt (deflated 63%)
  adding: content/eval_reports/eval_reports_report.txt (deflated 60%)
  adding: content/eval_reports/e5_연구계발계획서v6_report.txt (deflated 66%)


In [16]:
# 다운로드
from google.colab import files
files.download("my_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>